# Python/R 基础
---
本笔记由原笔记翻译而成, 其中包含部分补充笔记。
[原作者:github.com/rwbfd](https://github.com/rwbfd)

注意: 部分注释没有翻译。


## Python 基础
---
**请注意, 本笔记内容部分描述没有 Python Docs 中严谨。**
 




### 定义函数

请注意 type hint 不会进行强制类型转化。这与 Cython 等其他静态语言不一样。

In [ ]:
def myfunc(a:float, *args, **kwargs) -> str:
    return str(a)

In [ ]:
# 这看似不起作用,但是其实可以!
import numpy as np
x = np.array([1,1])
myfunc(x)

# 解释: 此处的a: float中的float并没有约束作用,
# 所以a 接受的实际类型是<class 'numpy.ndarray'>

`*args` 被称为 list unpacks。在函数内部, `args` 就像列表(lists)一样。

In [ ]:
def mysum(*args):
    result = 0
    for x in args:
        result += x
    return result
mysum(1,2,3) # Works
mysum(2,3,5,6) #Works

另一方面, `**kwargs` 被称为 keyword argument。其本质是一个 Python 字典变量。 

In [ ]:
def my_concat(**kwargs):
    result = ""
    
    for k, v in kwargs.items():
        result += v
    return result

In [ ]:
my_concat(x="a",y="b") # works
# 此处的 x, y 会被推断为<class 'str'>

In [ ]:
my_concat(fff = 1, bsr = 2) # not really
# 错误原因是因为 result = ""

In [ ]:
# 除此之外还有这种格式:
my_concat(**{"x": "a", "y": "b"}) # works

In [ ]:
my_concat(**{x: "a", y: "b"}) # not really
# 错误原因 似乎 是因为此时的 x, y 不会进行类型推断。
# 说 似乎 因为本人还没有探究清楚。

**注意:**
> 在 Python Docs 中指出:
>> 当存在一个形式为 `**name` 的最后一个形参时, 它会接收一个字典, 其中包含除了与已有形参相对应的关键字参数以外的所有关键字参数。
>>
>> 这可以与一个形式为 `*name`，接收一个包含除了与已有形参列表以外的位置参数的`元组`的形参组合使用 (`*name` 必须出现在 `**name` 之前)。

### 处理异常

---
处理异常最常用的方法是`触发异常`(也称为 `抛出`)。

**注释:** `Raising Exceptions` 在官方文档里被翻译为 `触发异常`, 此段翻译中**没有**精准的区别`触发(raise)`与`抛出(throw)`。

**基础处理异常代码**

> `try:` 执行代码, 注意`try:`不仅可以捕获`try:`代码块中的异常,还可以捕获`try:`之后函数中的异常。
> 
> `except:` 发生异常时执行的代码
> 
> `else:` 没有异常时执行的代码
> 
> `finally:` 不管有没有异常都会执行的代码



**注意:**详细使用应参见Python Docs。

**来自 Python Docs:**

`except` 子句可以在异常名称后面指定一个变量。这个变量和一个异常实例绑定, 它的参数存储在 `instance.args` 中。

为了方便起见, 异常实例定义了 `__str__()` , 因此可以直接打印参数而无需引用 `.args` 。也可以在抛出之前首先实例化异常, 并根据需要向其添加任何属性。

In [ ]:
# 来自 Python Docs 的示例代码:
try:
  raise Exception('spam', 'eggs')
except Exception as inst:
  print(type(inst))    # the exception instance
  print(inst.args)     # arguments stored in .args
  print(inst)          # __str__ allows args to be printed directly, but may be overridden in exception subclasses
  x, y = inst.args     # unpack args
  print('x =', x)
  print('y =', y)

<class 'Exception'>
('spam', 'eggs')
('spam', 'eggs')
x = spam
y = eggs


In [ ]:
def raise_exception(x):
    raise Exception("I am an EXCEPTION!!!") # Something bad has happend

def catcher(x):
    try:
        raise_exception(x) # This will run the code. If everything is fine, it will return normally.
    except (TypeError, NameError):  # If a specific error occurs, this will follow the behavior. 
        print("I am ok with this!")
    except Exception as e: # This is often used to handle unknown exception, by letting someone else do the job
        raise e
    finally: # This will always execute no matter what
        print("Let us swallow everything when exception occurs!")
    
    

In [ ]:
catcher(1)

这中方法有有几个问题.


*   只要有一个异常没有相应的处理，程序就会中断。
*   如果是我们是在测试代码, 那这是没问题的。但是如果是在生产实践中, 我们不推荐这么做。
*   一旦一个函数抛出异常, 调用该函数的其他所有人都必须通过添加`try-except`代码块来改进其代码。
*   许多异常将一直传递到顶部, 然后进行处理。 但是, 顶层函数不知道每个函数的详细信息! 因此, 设计一个完整的计划非常困难。



另一种方法是使用日志。 有许多记录日志的选项, 我们将不进行详细介绍。 习惯用法是记录何处出现了问题并指定行为。

好处是您将保持程序警告, 并通过调整日志级别来调整行为。 但是, **仍然需要有人处理异常!**

In [ ]:
import logging
logging.info("This is some useful information.")
logging.warning("This is some warning!")
logging.error("Something went wrong!")


最后且是非常流行的一个选择是使用Monad。 要去解释Monad将会非常复杂。 因此, 让我们来看一个例子。

**注释:** 

Monad: In functional programming, a monad is an abstraction that allows structuring programs generically.  ——  From Wiki  [Monad (functional programming)](https://en.wikipedia.org/wiki/Monad_(functional_programming))

In [ ]:
class Failure():
    def __init__(self, value, failed=False):
        self.value = value
        self.failed = failed
    def get(self):
        return self.value
    def is_failed(self):
        return self.failed
    def __str__(self):
        return ' '.join([str(self.value), str(self.failed)])
    def __or__(self, f):
        if self.failed:
            return self
        try:
            x = f(self.get())
            return Failure(x)
        except:
            return Failure(None, True)

In [ ]:
# This will work.
from operator import neg
x = '1'
y = Failure(x) | int | neg | str
print(y)

-1 False


In [ ]:
# This will not
from operator import neg
x = 'hahaha'
y = Failure(x) | int | neg | str
print(y)

None True


在[这里](https://github.com/jasondelaat/pymonad.git)可以找到一个很好的有关函数化编程合集。

使用`pip install PyMonad`命令进行安装。

**笔记:**

`__or__`函数解释:


> 参数解释: f 传入的是一个函数(函数名, 而不是函数结构)。

> 函数内容:
>
>> 首先检查是否还需要继续执行下一步, 如果之前有过报错( 反应在`self.failed`上 )则退出函数。
>
>> 尝试运行下一步, 如果报错交由`except:`处理, 并把`self.failed`标记为`True`。没有报错则将结果返回, 继续执行下一步。








### Python 类

In [ ]:
class MyClass(object):
    def __init__(self, x):
        self.x = x
    def __del__(self): # WARNING: Perhaps a very bad idea!
        print("I am gone")

In [ ]:
my_class = MyClass(1)

In [ ]:
del my_class

In [ ]:
my_class
# 这将会引发错误 NameError: name 'my_class' is not defined
# 因为 my_class 已经被 del
#
# 注意此行为与C++中delete或C中free()行为的区别:
# C或C++中这些函数的函数的操作对象是一个指针变量,
# 这意味这这个指针在被释放后会进入"吊起指针"的状态。
# 但本身指针的内容(不是指针所指向的内容)并未改变,
# 所以在C或C++编程中当释放一个指针之后应该将其赋NULL值。

In [ ]:
my_class_a = MyClass(1)
my_class_b = my_class_a
my_class_c = MyClass(1)

In [ ]:
my_class_b.x= 2
print(my_class_a.x) # Note that this is a reference to the class, therefore, they are pointing to the same thing which is why it changes. 

2


In [ ]:
my_class_b == my_class_a

True

In [ ]:
my_class_a = MyClass(1)
my_class_c = MyClass(1)
my_class_a == my_class_c

I am gone


False

In [ ]:
from copy import deepcopy
my_class_a = MyClass(1)
my_class_b = deepcopy(my_class_a)
my_class_b == my_class_a

I am gone
I am gone


False

In [ ]:
my_class_b.x= 2
print(my_class_a.x)

1


**补充笔记:有关 Python 垃圾回收 / 对象回收 / 引用计数 机制:**

In [ ]:
# 在搜集资料时, 有参考文件指出:
a = 40      # 创建对象  <40>
b = a       # 增加引用， <40> 的计数
c = [b]     # 增加引用.  <40> 的计数

del a       # 减少引用 <40> 的计数
b = 100     # 减少引用 <40> 的计数
c[0] = -1   # 减少引用 <40> 的计数

有趣的是, Python Docs中对有关信息的描述非常零散:


1.   在`Python/C API 参考手册`的`概述`的`对象、类型和引用计数`中有这样的描述:
    > 所有 Python 对象（甚至 Python 整数）都有一个 type 和一个 reference count。对象的类型确定它是什么类型的对象（例如整数、列表或用户定义函数；还有更多，如 标准类型层级结构 中所述）。对于每个众所周知的类型，都有一个宏来检查对象是否属于该类型；例如，当（且仅当） a 所指的对象是 Python 列表时 PyList_Check(a) 为真。

   更为重要的是, 在这之中有关`引用计数`的许多文档并未进行翻译。如果想进一步了解其工作原理应查阅相关文档原文。


2.   在`Python 语言参考手册`的`3. 数据模型`的`3.1. 对象、值与类型`中有这样的描述:
> 对象绝不会被显式地销毁；然而，当无法访问时它们可能会被作为垃圾回收。允许具体的实现推迟垃圾回收或完全省略此机制 --- 如何实现垃圾回收是实现的质量问题，只要可访问的对象不会被回收即可。

3.   在`Python/C API 参考手册`的`内存管理`中有对试图使用`malloc()`、 `calloc()`、 `realloc()` 和 `free()`的一些警示, 在不确定你的使用是否正确前应查看相应文档。


### 幽灵巴士事件(The Ghost Bus Incidence)

---
使用可变变量作为默认参数通常是一个可怕的主意。 以下片段表明了这一点。

In [ ]:
class GhostBus:
    def __init__(self, passengers=[]):
        self.passengers = passengers
    
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [ ]:
# 多运行这个代码块几次
ghost_bus = GhostBus()
ghost_bus.pick('A Ghost')
ghost_bus.passengers

['A Ghost']

为何会有这种错误发生呢? 请注意, `self.passengers`是对`passengers`的引用，而`passengers`又是对`[]`(一个全局变量)的引用。 请注意, 当你更改`self.passengers`的值时, 你也在同时修改了`[]`。 因此, 请改为使用`None`。

**注意:**

在Python Docs中对这个问题已经进行过提醒, 其指出:
> **重要警告:** 默认值只计算一次。默认值为列表、字典或类实例等可变对象时, 会产生与该规则不同的结果。例如, 下面的函数会累积后续调用时传递的参数:



In [ ]:
def f(a, L=[]):
    L.append(a)
    return L

print(f(1))
print(f(2))
print(f(3))

> 不想在后续调用之间共享默认值时, 应以如下方式编写函数:

In [ ]:
def f(a, L=None):
    if L is None:
        L = []
    L.append(a)
    return L

### 通用数据结构: 列表 (`list`)

---
Python 的 `list` 有一点像C++中的 `vector` (容器), 但它可以容纳任何类型的对象。 它是有序的。

**注释:**此处可能有语义歧义, 这里的“但它可以容纳任何类型的对象”表示的是Python 的 `list` 可以同时容纳不同类型的对象, 而C++中的 `vector`只能容纳单种变量或对象。C或C++惯用者应该会觉得这两者的区别巨大。

In [ ]:
a = []
# a = list()
b= [1,a,'2']

# 注意这两种初始化方式其实是有细微区别的:
# 区别主要在于list()是一个function call,
# Python的function call会创建stack, 
# 并且进行一系列参数检查的操作, 比较expensive, 
# 反观[]是一个内置的C函数, 可以直接被调用，因此效率高。

In [ ]:
b

[1, [], '2']

In [ ]:
b[0]

1

In [ ]:
b[:1]

[1]

In [ ]:
b[1:]

[[], '2']

In [ ]:
b[2:3]

['2']

In [ ]:
b[-1]

'2'

In [ ]:
b[:-2]

[1]

In [ ]:
b.append(5)
b

[1, [], '2', 5]

In [ ]:
b.extend([1,2])
b

[1, [], '2', 5, 1, 2]

In [ ]:
b.insert(1,'haha')
b

[1, 'haha', [], '2', 5, 1, 2]

In [ ]:
del b[0]
b

['haha', [], '2', 5, 1, 2]

In [ ]:
b.remove(1)

In [ ]:
matrix  = [[1,2],[3,4],[5,6],[7,8]]
matrix

[[1, 2], [3, 4], [5, 6], [7, 8]]

In [ ]:
tranpose =[[row[i] for row in matrix] for i in range(2)]

这里我们使用了列表推导式 `[x for x in iterable]` , 简而言之:

```
x = [i*2 for i in range(10]
```

和后者是一样的

```
x = list()
for i in range(10):
    x.append(i*2)
```

### 通用数据结构: 集合 (`set`)

---
其本质上是一个哈希集, 这基本上意味着它是无序的。 可以“等效”为C++中的`unordered_set`。 另外, 其中没有重复的元素。

In [ ]:
a = {1,2,3}

In [ ]:
my_set = {1, 3}
print(my_set)
my_set.add(2)
print(my_set)
my_set.update([2, 3, 4])
print(my_set)
my_set.update([4, 5], {1, 6, 8})
print(my_set)

{1, 3}
{1, 2, 3}
{1, 2, 3, 4}
{1, 2, 3, 4, 5, 6, 8}


In [ ]:
my_set.add(1)
my_set

{1, 2, 3, 4, 5, 6, 8}

In [ ]:
my_set.remove(1)
my_set

{2, 3, 4, 5, 6, 8}

In [ ]:
set_a = {1,2,3}
set_b = {3,4,5}

这里有一些集合的操作, 不言自明。 

In [ ]:
print(set_a|set_b)
print(set_a - set_b)
print(set_b - set_a)
print(set_a.union(set_b))
print(set_a.intersection(set_b))
print(set_a^set_b)

{1, 2, 3, 4, 5}
{1, 2}
{4, 5}
{1, 2, 3, 4, 5}
{3}
{1, 2, 4, 5}


### 通用数据结构: 字典 (`dict`)

---
Dict基本上是一个哈希表。可以“等效”为C++中的`unordered_map`。因此, 其是无序的。为了避免错误, 如果需要有序的, 请使用OrderedDict。 

**注意:**在 Python3.7+, 字典被确定为有序。 但为避免错误, 如果需要使用有序dict应另寻他法。

In [ ]:
a = dict()
a = {'x':'1', 'y':'2'}

In [ ]:
print(a['x'])
print(a['not_here'])

In [ ]:
a['new_element'] = 'haha'
print(a)

{'x': '1', 'y': '2', 'new_element': 'haha'}


In [ ]:
print(a.keys())
print(a.values())

In [ ]:
del a['new_element']

In [ ]:
a

In [ ]:
keys = ['a','b','c']
values = [1,2,3]
dict_from_zip = dict(zip(keys, values))
print(dict_from_zip)

{'a': 1, 'b': 2, 'c': 3}


In [ ]:
def my_concat(**kwargs):
    result = ""
    
    for k, v in kwargs.items():
        result += v
    return result
my_concat(x="a",y="b")

In [ ]:
my_concat(**a)

In [ ]:
# 你还可以使用字典推导式来缩短代码。
odd_squares = {x: x*x for x in range(11) if x % 2 == 1}
print(odd_squares)

{1: 1, 3: 9, 5: 25, 7: 49, 9: 81}


### 通用数据结构: 命名元组 (`NamedTuple`)

In [ ]:
from collections import namedtuple

In [ ]:
employee = namedtuple('Employee', ['age','place', 'education'])

In [ ]:
tom = employee(age=10, place='beijing', education='none')

In [ ]:
print(tom)

Employee(age=10, place='beijing', education='none')


### 通用数据结构: 数据类 (`dataclass`)

---

数据类是将多个参数传递给函数的好方法。它有助于进行文档和范围检查, 因此人们不会仅仅是将任何内容堆叠到其中。

In [ ]:
from dataclasses import dataclass, field
from typing import Optional

In [ ]:
@dataclass
class MyDataClass:
    name : str = field(
    default='tom',
    metadata={'help':"Name of the person"})
    
    age: Optional[int] = field(
    default = None,
    metadata={'help':"Age of the pesson. Optional."})
    
    vip: int = field(
    default = 100,
    metadata = {'help':"Some very important field."})
        

    def __post_init__(self): # This function will help you to handle illegal argument. 
        if self.vip <= 0:
            raise Exception("That important thing has to be larger than 0")
            
    @property
    def age_type(self):
        if self.age >= 100:
            return 'You are old'
        else:
            return 'You are still young' 

In [ ]:
my_data_class = MyDataClass(name='jerry', age = 20)
print(my_data_class)

MyDataClass(name='jerry', age=20, vip=100)


In [ ]:
print(my_data_class.age)
print(my_data_class.age_type)


20
You are still young


关于文档的那些事: 

通常, 使用[Spinx](https://www.sphinx-doc.org/en/master/)来生成文档是一个很好的主意。因此，应该给函数一些命令。 In general, for public api's, the docstring should include at least 通常, 对于公共API, 文档字符串应至少包含:

1.   功能;
2.   参数类型和说明;
3.   返回类型;
4.   用例(可选); 

请注意, 如果函数会更改某些输入参数。 **必须**在文档中突出显示此内容。


## R 基础

---
在进一步探究前, 让我们粗略地介绍R能做什么以及魔术函数。 要使用R，你必须在先在你的ipynb中激活它。

In [ ]:
%load_ext rpy2.ipython

为了使用R, 我们可以用`%%R`让这一个cell可以使用R。

In [ ]:
%%R # 像这样
install.packages('dplyr')

NULL


In [ ]:
%%R 
library('caret')

In [ ]:
%%R
# ->, <-, = 这些都是赋值语句
a <- 1
2 -> b
c = 1
a == c

In [ ]:
%%R
for (i in 1:100){
    print(i)
}

In [ ]:
%%R
myfunc <- function(a){
    a = a+1
    return(a+1)
}

In [ ]:
%%R
myfunc(a) # It will usually make a copy

[1] 3


In [ ]:
%%R
a

[1] 1


In [ ]:
%%R
data(mtcars) # This is a built-in R dataset

In [ ]:
%%R
summary(mtcars)

      mpg             cyl             disp             hp       
 Min.   :10.40   Min.   :4.000   Min.   : 71.1   Min.   : 52.0  
 1st Qu.:15.43   1st Qu.:4.000   1st Qu.:120.8   1st Qu.: 96.5  
 Median :19.20   Median :6.000   Median :196.3   Median :123.0  
 Mean   :20.09   Mean   :6.188   Mean   :230.7   Mean   :146.7  
 3rd Qu.:22.80   3rd Qu.:8.000   3rd Qu.:326.0   3rd Qu.:180.0  
 Max.   :33.90   Max.   :8.000   Max.   :472.0   Max.   :335.0  
      drat             wt             qsec             vs        
 Min.   :2.760   Min.   :1.513   Min.   :14.50   Min.   :0.0000  
 1st Qu.:3.080   1st Qu.:2.581   1st Qu.:16.89   1st Qu.:0.0000  
 Median :3.695   Median :3.325   Median :17.71   Median :0.0000  
 Mean   :3.597   Mean   :3.217   Mean   :17.85   Mean   :0.4375  
 3rd Qu.:3.920   3rd Qu.:3.610   3rd Qu.:18.90   3rd Qu.:1.0000  
 Max.   :4.930   Max.   :5.424   Max.   :22.90   Max.   :1.0000  
       am              gear            carb      
 Min.   :0.0000   Min.   :3.000  

In [ ]:
%%R
# $相当我们常见语言中的 . 
# R中的 . 是合法命名符号, 
# 其含义与常见语言中_相似
mtcars$mpg

 [1] 21.0 21.0 22.8 21.4 18.7 18.1 14.3 24.4 22.8 19.2 17.8 16.4 17.3 15.2 10.4
[16] 10.4 14.7 32.4 30.4 33.9 21.5 15.5 15.2 13.3 19.2 27.3 26.0 30.4 15.8 19.7
[31] 15.0 21.4


# Python对象中基本定制

---
注释: 此章原标题为“Magic Functions in Python Object”, 译为“Python对象中的魔法函数”。但其内容是Python Docs中的`3.3.1. Basic customization`, 官方译为:`3.3.1. 基本定制`。

In [ ]:
class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

让我们看看是否有更好地办法来输出。

In [ ]:
class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __repr__(self):
        return 'Vector(%r,%r)' % (self.x, self.y)
    def __str__(self):                              
        return 'Vector(%r,%r)' % (self.x, self.y)

In [ ]:
v = Vector(1,2)
print(str(v))
print(v)

Vector(1,2)
Vector(1,2)


以下是部分运算符。

In [ ]:
class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __repr__(self):
        return 'Vector(%r,%r)' % (self.x, self.y)
    
    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)
    
    def __sub__(self, other):
        x = self.x - other.x
        y = self.y - other.y
        return Vector(x, y)
    
    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

In [ ]:
v1 = Vector(0,0)
v2 = Vector(1,2)

v1 *= 2

以下是部分比较符。

In [ ]:
from math import hypot

class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __repr__(self):
        return 'Vector(%r,%r)' % (self.x, self.y)
    
    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)
    
    def __sub__(self, other):
        x = self.x - other.x
        y = self.y - other.y
        return Vector(x, y)
    
    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)
    
    def __abs__(self):
        return hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __lt__(self, other):
        return abs(self) < abs(other)
    
    def __gt__(self, other):
        return abs(self) > abs(other)

In [ ]:
v1 = Vector(1,1)
v2 = Vector(1,1)
v3 = Vector(1,2)

print(v1 == v2)
print(v1 == v3)

print(v3 > v1)
print(v1 < v3)

True
False
True
True


# Python中函数化编程基础

### 常见高阶函数

In [ ]:
my_input = [1,2,3,4,5,6,6]
result = map(lambda x: x+1, my_input)
print(result) # map is lazy
print(list(result))

[2, 3, 4, 5, 6, 7, 7]


In [ ]:
# 注意:
# 此处如果按照通常的思路去处理, 需要三次for循环.
# 但是其实这三个步骤实际只用了一个for循环的时间.
from functools import reduce
result = reduce(lambda x, y: x+y, filter(lambda x: x > 3, map(lambda x: x+1, my_input)))

In [ ]:
print(result)

29


### 装饰器

In [ ]:
def my_decorator(func):
    def my_decorator_impl(x):
        result = x if x > 0 else 0
        return func(result)
    return my_decorator_impl

@my_decorator
def myfunc(x):
    return np.sqrt(x)

In [ ]:
# This is the same
new_func = my_decorator(myfunc)


0.0

In [ ]:
myfunc(-1)

0.0

In [ ]:
from functools import partial
def decor_impl(fun, argument):
    def impl(x):
        result = x if x > argument else argument
        return fun(result)
    return impl

decor = partial(decor_impl, argument = 2)

@decor
def myfunc(x):
    return np.sqrt(x)

In [ ]:
myfunc(-1)

1.4142135623730951

In [ ]:
def para(dec):
    def layer(*args, **kwargs):
        def repl(f):
            return dec(f, *args, **kwargs)
        return repl
    return layer

@para
def decor(f, n):
    def impl(x):
        result = x if x > n else n
        return f(result)
    return impl

@decor(0)
def myfunc(x):
    return np.sqrt(x)

In [ ]:
def para(dec):
    def layer(*args, **kwargs):
        def repl(f):
            return dec(f, *args, **kwargs)
        return repl
    return layer

def decor(f, n):
    def impl(x):
        result = x if x > n else n
        return f(result)
    return impl